In [2]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# global variables
numb_steps = 5 # number of truncated backprop steps
batch_size = 200
num_classes = 2
state_size = 4
learning_rate = 0.1

Dependencies on input at step t-3 and t-8

How does gen_data work?

How does gen_batch work?

In [4]:
def gen_data(size=1000000):
    
    X = np.array(np.random.choice(2, size=(size,)))
    Y = []
    
    for i in range(size):
        threshold = 0.5
        if X[i-3] == 1:
            threshold += 0.5
        if X[i-8] == 1:
            threshold += 0.5
        if X[i-3] == 1:
            threshold += 0.5
        if np.random.rand() > threshold:
            Y.append(0)
        else:
            Y.append(1)
    
    return X, np.array(y)

# adapted from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/ptb/reader.py
def gen_batch(raw_data, batch_size, num_steps):
    raw_x, raw_y = raw_data
    data_length = len(raw_x)
    
    # partition raw data into batches and stack them vertically in a data matrix
    batch_partition_length = data_length // batch_size # don't want partial sized batches
    data_x = np.zeros([batch_size, batch_partition_length], dtype=np.int32)
    data_y = np.zeros([batch_size, batch_partition_length], dtype=np.int32)
    for i in range(batch_size):
        data_x[i] = raw_x[batch_partition_length * i:batch_partition_length * (i + 1)]
        data_y[i] = raw_x[batch_partition_length * i:batch_partition_length * (i + 1)]

    # futher divide batch partitions into num_steps for truncated backprop
    epoch_size = batch_partition_length // num_steps
    
    for i in range(epoch_size):
        x = data_x[:, i * num_steps:(i + 1) * num_steps]
        y = data_y[:, i * num_steps:(i + 1) * num_steps]
        yield (x, y)

def gen_epochs(n, num_steps):
    for i in range(n):
        yield gen_batch(gen_data(), batch_size, num_steps)